# Data import
This notebook contains imports of the data, namely, players, their urls and guesses of other players of the game.


In [37]:
import pandas as pd
from pathlib import Path

NB: Don't forget to run all the scripts prior to this step.

In [38]:
# initialize all dataframes
df_raw = pd.DataFrame()
df2_raw = pd.DataFrame()

In [39]:
# Load all inputs
#ans_file = Path('../data/answers.csv')
ans_file = Path('../data/real-guesses.csv')
#gt_file = Path('../data/ground_truth.csv')
gt_file = Path('../data/input-gt.csv')
if ans_file.is_file():
    df_raw = pd.read_csv(ans_file,error_bad_lines=False)
if gt_file.is_file():
    df2_raw = pd.read_csv(gt_file,error_bad_lines=False,skipinitialspace=True).apply(lambda s:s.str.replace('"', ''))


In [40]:
#check
#print(df_raw.shape, df2_raw.shape)
del df_raw['Timestamp']
df_raw.head()



,Current URL,Player,Guess
0,https://www.youtube.com/watch?v=T5tGEv9cegY,Ali Huseynov,Фуад
1,https://www.youtube.com/watch?v=T5tGEv9cegY,Rena B.,Фуад
2,https://www.youtube.com/watch?v=T5tGEv9cegY,Rena A,Фуад
3,https://www.youtube.com/watch?v=T5tGEv9cegY,Фуад,Rena B.
4,https://www.youtube.com/watch?v=T5tGEv9cegY,Kamusik,Фуад


In [41]:
# change column titles
df2_raw.rename(columns = {df2_raw.columns[1]:'Player'},inplace=True)
df2_raw.rename(columns = {df2_raw.columns[2]:'URL'},inplace=True)
del df2_raw['Timestamp']
#df2_raw.columns.str.lstrip()
df2_raw.head()
#df_raw.shape

,Player,URL
0,Rena B.,https://www.youtube.com/watch?v=6ybd5rbQ5rU
1,Aslan,https://www.youtube.com/watch?v=yawlrsRDabA
2,Ali Huseynov,https://youtu.be/J3eeh15GXj0
3,Ирада,https://www.youtube.com/watch?v=T5tGEv9cegY
4,Girey,https://www.youtube.com/watch?v=9pBmkOB1P8U


In [42]:
# which urls
urls = df_raw['Current URL'].unique()
#print(urls.tolist())

# which players
players = df_raw['Player'].unique()

# which guesses
guess = df_raw['Guess'].unique()

In [43]:
df_player_guess=df_raw.loc[:,['Player','Guess']]
df_player_guess.head()

,Player,Guess
0,Ali Huseynov,Фуад
1,Rena B.,Фуад
2,Rena A,Фуад
3,Фуад,Rena B.
4,Kamusik,Фуад


In [44]:
# looking at the guesses per URL slice
df_url_guess=df_raw.loc[:,['Current URL','Guess']] #only guesses per URL
df_url_guess.head()

,Current URL,Guess
0,https://www.youtube.com/watch?v=T5tGEv9cegY,Фуад
1,https://www.youtube.com/watch?v=T5tGEv9cegY,Фуад
2,https://www.youtube.com/watch?v=T5tGEv9cegY,Фуад
3,https://www.youtube.com/watch?v=T5tGEv9cegY,Rena B.
4,https://www.youtube.com/watch?v=T5tGEv9cegY,Фуад


In [45]:
# code from https://towardsdatascience.com/from-dataframe-to-network-graph-bbb35c8ab675
# function creating leaderboard

def create_leaderboard(df,col_name, tab_header):
    leaderboard = df[col_name].value_counts(ascending=True)
    s = pd.Series(leaderboard, name=tab_header)
    df2 = s.to_frame().sort_values(tab_header, ascending=False)
    return df2

In [46]:
# total number of votes per link
votes_links = create_leaderboard(df_raw,'Current URL','votes')

In [47]:
# total votes for each player
total_votes = create_leaderboard(df_raw,'Guess','votes')
total_votes = total_votes.sort_values("votes", ascending=False).reset_index()
total_votes.rename(columns = {'index':'Player'}, inplace = True)
#total_votes['votes'] = pd.to_numeric(total_votes['votes'])

In [48]:
# stats of guesses per URL

guess_stats = df_raw.groupby(by=['Current URL', 'Guess'])['Guess'].count()
s = pd.Series(guess_stats, name="votes")
df_guess = s.to_frame().sort_values("votes", ascending=False).reset_index()
